In [1]:
import numpy as np

# Constants
pos = np.array([0.580303, -3.68339, 3.44946])
scale = np.array([-4.44527, -4.37531, -5.52493])
rot = np.array([0.666832, 0.0965957, -0.328523, 0.0227409])
focal_length = 1.0
resolution = np.array([1920, 1080])

# Functions
def compute_cov3d(scale, rotation):
    # Create scaling matrix
    S = np.diag(scale)

    # Normalize quaternion
    r, x, y, z = rotation

    # Compute rotation matrix from quaternion
    R = np.array([
        [1.0 - 2.0 * (y**2 + z**2), 2.0 * (x * y - r * z), 2.0 * (x * z + r * y)],
        [2.0 * (x * y + r * z), 1.0 - 2.0 * (x**2 + z**2), 2.0 * (y * z - r * x)],
        [2.0 * (x * z - r * y), 2.0 * (y * z + r * x), 1.0 - 2.0 * (x**2 + y**2)]
    ])

    # Compute the transformation matrix
    M = S @ R

    # Compute 3D covariance matrix
    Sigma = M.T @ M

    # Return the covariance matrix (upper triangular matrix)
    return np.array([
        [Sigma[0, 0], Sigma[0, 1], Sigma[0, 2]],
        [0.0, Sigma[1, 1], Sigma[1, 2]],
        [0.0, 0.0, Sigma[2, 2]]
    ])

def compute_cov2d(pos, focal_length, resolution, cov3d, view_matrix):
    # Calculate aspect ratio
    aspect_ratio = resolution[0] / resolution[1]

    # Compute focal_x and focal_y
    focal_x = focal_length * aspect_ratio
    focal_y = focal_length

    # Transform the point to view space
    t = view_matrix @ np.append(pos, 1.0)
    t_view = t[:3] / t[3]

    # Compute Jacobian matrix
    J = np.array([
        [focal_x / t_view[2], 0.0, -(focal_x * t_view[0]) / (t_view[2]**2)],
        [0.0, focal_y / t_view[2], -(focal_y * t_view[1]) / (t_view[2]**2)],
        [0.0, 0.0, 0.0]
    ])

    # Compute 2D covariance matrix
    W = view_matrix[:3, :3]
    T = W @ J

    cov = T.T @ cov3d @ T

    # Apply low-pass filter
    cov[0, 0] += 0.3
    cov[1, 1] += 0.3

    # Return the 2D covariance matrix (diagonal matrix)
    return np.array([cov[0, 0], cov[0, 1], cov[1, 1]])

def world_to_pixel(pos, camera, W, H):
    p_view = camera['view'] @ np.append(pos, 1.0)
    p_hom = camera['projection'] @ p_view
    p_w = 1.0 / max(p_hom[3], 1e-7)
    p_proj = np.array([p_hom[0] * p_w, p_hom[1] * p_w, p_hom[2] * p_w])

    return np.array([
        (p_proj[0] * 0.5 + 0.5) * W,
        (p_proj[1] * 0.5 + 0.5) * H
    ])

# Example view and projection matrices (identity for simplicity)
camera = {
    'view': np.eye(4),
    'projection': np.eye(4),
    'resolution': resolution,
    'focal_length': focal_length
}

# Compute 3D and 2D covariance matrices
cov3d = compute_cov3d(scale, rot)
cov2d = compute_cov2d(pos, focal_length, resolution, cov3d, camera['view'])

# Invert 2D covariance matrix
det = cov2d[0] * cov2d[2] - cov2d[1]**2
if det != 0:
    det_inv = 1.0 / det
    conic = np.array([cov2d[2] * det_inv, -cov2d[1] * det_inv, cov2d[0] * det_inv])

    # Compute eigenvalues and radius
    mid = 0.5 * (cov2d[0] + cov2d[2])
    lambda1 = mid + np.sqrt(max(0.1, mid**2 - det))
    lambda2 = mid - np.sqrt(max(0.1, mid**2 - det))
    radius = np.ceil(3.0 * np.sqrt(max(lambda1, lambda2)))

    print("Radius:", radius)

# Compute pixel coordinates
pixel_coord = world_to_pixel(pos, camera, resolution[0], resolution[1])
print("Pixel Coordinates:", pixel_coord)


Result: [111.41111111  11.11111111 222.52222222]
